<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/TrajectoryOfSphereWithSpin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

In [108]:
def Cd(Re):
    a=Re/5
    b=Re/2.63e5
    c=Re/1e6
    return 24/Re + 2.6*a/(1+a**1.52) +0.411*b**(-9)/(1+b**(-9)) + 0.25*c/(1+c)

In [109]:
x = np.logspace(2,7,500)
Cd_vals = Cd(x)

In [110]:
fig = make_subplots()
fig.add_scatter(x=x,y=Cd_vals, mode='lines')
fig.update_xaxes(type='log')
fig.update_layout(width=800,height=600, template='plotly_dark')

In [119]:
rho = 1.29 #kg/m3
mu = 1.8e-5
d = 0.22
A = np.pi*(d**2)/4
g = 9.81
m= 0.42
s= np.array([0,0,50])
s_mag = np.linalg.norm(s)


def rhs(t, xv):
    v=xv[3:]
    v_mag = np.linalg.norm(v)
    Re = rho*v_mag*d/mu
    Cl=0.2
    dxdt = v
    dvdt = np.array([0, 0, -g]) - rho*Cd(Re)*A*v_mag/2*v/m + rho*Cl*A* d/2 * np.cross(s,v)/m
    return np.concatenate([dxdt,dvdt])

def event(t, xv):
    return xv[2]

event.terminal=True
event.direction=-1

v_mag_mph = 80
angle = 30
side_angle = 15
v_mag = v_mag_mph * 0.44704
theta = angle*np.pi/180
phi = side_angle*np.pi/180
v = v_mag*np.array([np.cos(theta)*np.sin(phi), np.cos(theta)*np.cos(phi),  np.sin(theta)])
tend=10
res = solve_ivp(rhs, (0,tend), np.concatenate([[0,0,0],v]), method='Radau', dense_output=True, events=event)

In [122]:
t_plot = np.linspace(0,res.t_events[0][0],50)
x,y,z=res.sol(t_plot)[:3]
fig=make_subplots(rows=1,cols=1,specs=[[{'type':'surface'}]])
fig.add_scatter3d(x=x,y=y,z=z,mode='markers',row=1,col=1, marker=dict(
            color='rgba(0,0,200,0.5)',
            size=5,
            line=dict(
                color='rgba(0,0,200,0.5)',
                width=1
            )
        ),)
fig.update_layout(width=800,height=800,template='plotly_dark', scene_camera_eye=dict(x=0.1, y=-2, z=1),
                      scene = dict(xaxis = dict(nticks=10, range=[-15,15]),
                     yaxis = dict(nticks=10, range=[0,100]),
                     zaxis = dict(nticks=10, range=[0,50])))